In [ ]:
import ipython_importer
from _loadData import loadData
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Timeline, Grid, Bar
from pyecharts.commons.utils import JsCode
import pandas as pd
import numpy as np
def main(code, freq, speed):
    test1 = loadData(code, freq)
    data_use = test1.df_concat
    bg_color_js = """
    new echarts.graphic.RadialGradient(0.3, 0.3, 0.8, [{
            offset: 0,
            color: '#f7f8fa'
        }, {
            offset: 1,
            color: '#cdd0d5'
        }])"""
        
    timeline = Timeline(
        init_opts=opts.InitOpts(
            bg_color=JsCode(bg_color_js),
            width='980px',
            height='600px'))
    timeline.add_schema(
        is_auto_play=True,  # 自动播放
        is_loop_play=False,  # 循环播放
        is_timeline_show=True,
        play_interval=2000 / speed,
    )

    x_data = [str(i) for i in data_use.index]
    record=0
    data = np.array(data_use[["open", "close", "low", "high"]]).tolist()
    for index, row in data_use.iterrows():
        record+=1
        kline = (
            Kline(init_opts=opts.InitOpts(
                    animation_opts=opts.AnimationOpts(animation_easing="linear")))
            .add_xaxis([str(i) for i in data_use.index])
            .add_yaxis(
                "Stock Info",
                data[:record],
                itemstyle_opts=opts.ItemStyleOpts(
                    color="#ef232a",
                    color0="#14b143",
                    border_color="#ef232a",
                    border_color0="#14b143",
                ),
            )
            .set_global_opts(
                title_opts=opts.TitleOpts(title=code + " " + freq + " 走势图", pos_left="0"),
                xaxis_opts=opts.AxisOpts(
                    type_="category",
                    is_scale=True,
                    boundary_gap=False,
                    axisline_opts=opts.AxisLineOpts(is_on_zero=False),
                    splitline_opts=opts.SplitLineOpts(is_show=False),
                    split_number=20,
                    min_="dataMin",
                    max_="dataMax",
                ),
                yaxis_opts=opts.AxisOpts(
                    is_scale=True, splitline_opts=opts.SplitLineOpts(is_show=True)
                ),
                tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="line"),
                datazoom_opts=[
                    opts.DataZoomOpts(
                        is_show=False, type_="inside", xaxis_index=[0, 0], range_start=0, range_end=100
                    ),
                    opts.DataZoomOpts(
                        is_show=True, type_="inside",xaxis_index=[0, 1], pos_top="97%",  range_start=0, range_end=100
                    ),
                    ],
                )
            )
        kline_line = (
            Line()
            .add_xaxis([str(i) for i in data_use.index])
            .add_yaxis(
                series_name="MA5",
                y_axis=data_use["MA5"][:record].tolist(),
                is_smooth=True,
                linestyle_opts=opts.LineStyleOpts(opacity=0.5),
                label_opts=opts.LabelOpts(is_show=False),
            )
            .add_yaxis(
                series_name="MA10",
                y_axis=data_use["MA10"][:record].tolist(),
                is_smooth=True,
                linestyle_opts=opts.LineStyleOpts(opacity=0.5),
                label_opts=opts.LabelOpts(is_show=False),
            )
            .add_yaxis(
                series_name="MA20",
                y_axis=data_use["MA20"][:record].tolist(),
                is_smooth=True,
                linestyle_opts=opts.LineStyleOpts(opacity=0.5),
                label_opts=opts.LabelOpts(is_show=False),
            )
            .add_yaxis(
                series_name="MA30",
                y_axis=data_use["MA30"][:record].tolist(),
                is_smooth=True,
                linestyle_opts=opts.LineStyleOpts(opacity=0.5),
                label_opts=opts.LabelOpts(is_show=False),
            )
            .set_global_opts(
                xaxis_opts=opts.AxisOpts(
                    type_="category",
                    grid_index=1,
                    axislabel_opts=opts.LabelOpts(is_show=False),
                ),
                yaxis_opts=opts.AxisOpts(
                    grid_index=1,
                    split_number=3,
                    axisline_opts=opts.AxisLineOpts(is_on_zero=False),
                    axistick_opts=opts.AxisTickOpts(is_show=False),
                    splitline_opts=opts.SplitLineOpts(is_show=False),
                    axislabel_opts=opts.LabelOpts(is_show=True),
                ),
            )
        )
        overlap_kline_line = kline.overlap(kline_line)

        bar_1 = (
            Bar()
            .add_xaxis([str(i) for i in data_use.index])
            .add_yaxis(
                series_name="Volume",
                y_axis=data_use["volume"][:record].tolist(),
                xaxis_index=1,
                yaxis_index=1,
                label_opts=opts.LabelOpts(is_show=False),
            )
            .set_global_opts(
                xaxis_opts=opts.AxisOpts(
                    type_="category",
                    grid_index=1,
                    axislabel_opts=opts.LabelOpts(is_show=False),
                ),
                legend_opts=opts.LegendOpts(is_show=False),
            )
        )
        grid_chart = Grid(init_opts=opts.InitOpts(width="1000px", height="800px"))
        grid_chart.add(
            overlap_kline_line,
            grid_opts=opts.GridOpts(pos_left="3%", pos_right="1%", height="60%"),
        )
        grid_chart.add(
        bar_1,
        grid_opts=opts.GridOpts(pos_left="3%", pos_right="1%",pos_top="71%", height="10%"),
            )
        timeline.add(grid_chart, str(index))
    return timeline
stockCode = input("请输入要查询的股票代码(例如000001.SZ): ")
speed = int(input("请输入行情回放速度(整数类型，1~10)"))
freq = "180S"
timeline = main(stockCode, freq, speed)
timeline.render("../demo/echartsAction-" + stockCode + " .html")
timeline.render_notebook()